In [22]:
from hashlib import md5
import pandas as pd
import json
import numpy as np
import geopandas as gpd
from random import randint, random, choice, uniform
from datetime import datetime

from shapely.ops import cascaded_union
from shapely.geometry import Point

In [2]:
def generate_random(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    counter = 0
    while True:
        pnt = Point(uniform(minx, maxx), uniform(miny, maxy))
        if polygon.contains(pnt):
            return pnt


# This needs to be updated based on a lookup from CSIRO
def gridcell_to_loc(row):
    point = generate_random(eth)
    row['latitude'] = point.y
    row['longitude'] = point.x
    return row

In [3]:
admin2 = gpd.read_file("../CSIRO-Integration/gadm36_ETH_shp/gadm36_ETH_2.shp")
admin2['country'] = admin2['NAME_0']
admin2['state'] = admin2['NAME_1']
admin2['admin1'] = admin2['NAME_1']
admin2['admin2'] = admin2['NAME_2']
admin2['GID_2'] = admin2['GID_2'].apply(lambda x: x.split("_")[0])
admin2['GID_1'] = admin2['GID_1'].apply(lambda x: x.split("_")[0])

eth = cascaded_union(admin2.geometry)

In [32]:
rainfall = ['low','medium','high']
temperature = ['low','medium','high']

for rain in rainfall:
    
    for temp in temperature:
        
        objs = []
        for i in range(1,1000):
            obj = {}
            obj['datetime'] = datetime(year=2018, month=randint(1,12), day=1).strftime("%Y-%m-%d")
            obj['timber_production'] = randint(1*10**5, 3*10**6) # kilograms
            obj['cereal_production'] = randint(1*10**5, 3*10**7) # kilograms
            objs.append(obj)
        
        df = pd.DataFrame(objs)
        df['rainfall'] = rain
        df['temperature'] = temp
        df = df.apply(lambda row: gridcell_to_loc(row), axis=1)
        
        df.to_csv(f"example_runs/{rain}_rainfall__{temp}_temperature.csv", index=False)